In [11]:
from dotenv import load_dotenv
from kubernetes import client
from openshift.dynamic import DynamicClient
from openshift.helper.userpassauth import OCPLoginConfiguration
import os
import pandas as pd
import requests
import urllib3

load_dotenv() 

# disable pesky urllib3 ssl warnings.
urllib3.disable_warnings()

OCP_APIHOST = os.environ["OCP_APIHOST"]
OCP_USERNAME = os.environ["OCP_USERNAME"]
OCP_PASSWORD = os.environ["OCP_PASSWORD"]
QPC_BASE_URL = os.environ["QPC_BASE_URL"]
QPC_USERNAME = os.environ["QPC_USERNAME"]
QPC_PASSWORD = os.environ["QPC_PASSWORD"]
QPC_REPORT_ID = os.environ["QPC_REPORT_ID"]
OUTPUT_DIR = os.environ["OUTPUT_DIR"]

kubeConfig = OCPLoginConfiguration(ocp_username=OCP_USERNAME, ocp_password=OCP_PASSWORD)
kubeConfig.host = OCP_APIHOST
kubeConfig.verify_ssl = False

kubeConfig.get_token()

 
k8s_client = client.ApiClient(kubeConfig)
 
dyn_client = DynamicClient(k8s_client)
# v1_projects = dyn_client.resources.get(api_version='project.openshift.io/v1', kind='Project')
# project_list = v1_projects.get()

In [12]:
routes_client = dyn_client.resources.get(api_version='route.openshift.io/v1', kind='Route')
routes = routes_client.get(namespace="openshift-monitoring", field_selector="metadata.name=prometheus-k8s").items
prometheus_host = routes[0]["spec"]["host"] + routes[0]["spec"].get("path", "/api")
prometheus_host

'prometheus-k8s-openshift-monitoring.apps.sharedocp4upi411ovn.lab.upshift.rdu2.redhat.com/api'

In [13]:
console_client = dyn_client.resources.get(
    api_version="config.openshift.io/v1", kind="Console"
)
console_url = console_client.get().items[0]["status"]["consoleURL"]
print(console_url)

# prometheus_query="node_role_os_version_machine:cpu_capacity_sockets:sum"
#prometheus_query="workload:capacity_physical_cpu_cores:sum" # https://github.com/openshift/cluster-monitoring-operator/blob/1c595854ab4acbefd70446b54ae314016f400d8e/jsonnet/rules.libsonnet#L249-L251
# prometheus_query="node_role_os_version_machine:cpu_capacity_cores:sum" # https://github.com/openshift/cluster-monitoring-operator/blob/f365468055303edc8a4bb93e494bcbdedcf587bd/manifests/0000_50_cluster-monitoring-operator_04-config.yaml#L248-L253
# slightly modified version of what cluster-monitoring-operator uses for getting cpu_capacity_sockets
prometheus_query = '''
count by(instance, label_node_hyperthread_enabled, label_node_openshift_io_os_id, label_node_role_kubernetes_io_master, label_node_role_kubernetes_io_infra) 
(max 
    by(node, instance, package, label_node_hyperthread_enabled, label_node_openshift_io_os_id, label_node_role_kubernetes_io_master, label_node_role_kubernetes_io_infra)
    (cluster:cpu_core_node_labels)
)
'''
# other interesting metrics
# - node_cpu_info
# - machine_cpu_sockets
# - machine_cpu_cores
# - machine_cpu_physical_cores
# - node_network_info (macaddresses)
# - cluster_infrastructure_provider

# r.status_code, r.json()

def prometheus_json_to_df(query, value_key):
    response = requests.get(
        f"https://{prometheus_host}/v1/query?query={query}",
        verify=False,
        headers=kubeConfig.api_key,
    )
    assert response.ok
    return pd.DataFrame(
        {value_key:d["value"][1], **d["metric"]}
        for d in response.json()["data"]["result"]
    )

prometheus_json_to_df(prometheus_query, "sockets")


https://console-openshift-console.apps.sharedocp4upi411ovn.lab.upshift.rdu2.redhat.com


,sockets,instance,label_node_hyperthread_enabled,label_node_openshift_io_os_id,label_node_role_kubernetes_io_master
0,4,master-0.sharedocp4upi411ovn.lab.upshift.rdu2....,false,rhcos,true
1,4,master-1.sharedocp4upi411ovn.lab.upshift.rdu2....,false,rhcos,true
2,4,master-2.sharedocp4upi411ovn.lab.upshift.rdu2....,false,rhcos,true
3,4,worker-0.sharedocp4upi411ovn.lab.upshift.rdu2....,false,rhcos,NaN
4,4,worker-1.sharedocp4upi411ovn.lab.upshift.rdu2....,false,rhcos,NaN
5,4,worker-2.sharedocp4upi411ovn.lab.upshift.rdu2....,false,rhcos,NaN


In [14]:
prometheus_json_to_df("node_cpu_info", "?")

,?,__name__,cachesize,container,core,cpu,endpoint,family,instance,job,microcode,model,model_name,namespace,package,pod,service,stepping,vendor
0,1,node_cpu_info,512 KB,kube-rbac-proxy,0,0,https,23,master-0.sharedocp4upi411ovn.lab.upshift.rdu2....,node-exporter,0x1000065,1,AMD EPYC Processor (with IBPB),openshift-monitoring,0,node-exporter-sq5qm,node-exporter,2,AuthenticAMD
1,1,node_cpu_info,512 KB,kube-rbac-proxy,0,0,https,23,master-1.sharedocp4upi411ovn.lab.upshift.rdu2....,node-exporter,0x1000065,1,AMD EPYC Processor (with IBPB),openshift-monitoring,0,node-exporter-sbmqf,node-exporter,2,AuthenticAMD
2,1,node_cpu_info,512 KB,kube-rbac-proxy,0,0,https,23,master-2.sharedocp4upi411ovn.lab.upshift.rdu2....,node-exporter,0x1000065,1,AMD EPYC Processor (with IBPB),openshift-monitoring,0,node-exporter-2vh2b,node-exporter,2,AuthenticAMD
3,1,node_cpu_info,512 KB,kube-rbac-proxy,0,0,https,23,worker-0.sharedocp4upi411ovn.lab.upshift.rdu2....,node-exporter,0x1000065,1,AMD EPYC Processor (with IBPB),openshift-monitoring,0,node-exporter-tcq6s,node-exporter,2,AuthenticAMD
4,1,node_cpu_info,512 KB,kube-rbac-proxy,0,0,https,23,worker-1.sharedocp4upi411ovn.lab.upshift.rdu2....,node-exporter,0x1000065,1,AMD EPYC Processor (with IBPB),openshift-monitoring,0,node-exporter-k5d5x,node-exporter,2,AuthenticAMD
5,1,node_cpu_info,512 KB,kube-rbac-proxy,0,0,https,23,worker-2.sharedocp4upi411ovn.lab.upshift.rdu2....,node-exporter,0x1000065,1,AMD EPYC Processor (with IBPB),openshift-monitoring,0,node-exporter-bmnhv,node-exporter,2,AuthenticAMD
6,1,node_cpu_info,512 KB,kube-rbac-proxy,0,1,https,23,master-0.sharedocp4upi411ovn.lab.upshift.rdu2....,node-exporter,0x1000065,1,AMD EPYC Processor (with IBPB),openshift-monitoring,1,node-exporter-sq5qm,node-exporter,2,AuthenticAMD
7,1,node_cpu_info,512 KB,kube-rbac-proxy,0,1,https,23,master-1.sharedocp4upi411ovn.lab.upshift.rdu2....,node-exporter,0x1000065,1,AMD EPYC Processor (with IBPB),openshift-monitoring,1,node-exporter-sbmqf,node-exporter,2,AuthenticAMD
8,1,node_cpu_info,512 KB,kube-rbac-proxy,0,1,https,23,master-2.sharedocp4upi411ovn.lab.upshift.rdu2....,node-exporter,0x1000065,1,AMD EPYC Processor (with IBPB),openshift-monitoring,1,node-exporter-2vh2b,node-exporter,2,AuthenticAMD
9,1,node_cpu_info,512 KB,kube-rbac-proxy,0,1,https,23,worker-0.sharedocp4upi411ovn.lab.upshift.rdu2....,node-exporter,0x1000065,1,AMD EPYC Processor (with IBPB),openshift-monitoring,1,node-exporter-tcq6s,node-exporter,2,AuthenticAMD


In [15]:
prometheus_json_to_df("node_network_info", "?")

,?,__name__,address,broadcast,container,device,endpoint,instance,job,namespace,operstate,pod,service,duplex
0,1,node_network_info,00:00:00:00:00:00,00:00:00:00:00:00,kube-rbac-proxy,lo,https,master-0.sharedocp4upi411ovn.lab.upshift.rdu2....,node-exporter,openshift-monitoring,unknown,node-exporter-sq5qm,node-exporter,NaN
1,1,node_network_info,00:00:00:00:00:00,00:00:00:00:00:00,kube-rbac-proxy,lo,https,master-1.sharedocp4upi411ovn.lab.upshift.rdu2....,node-exporter,openshift-monitoring,unknown,node-exporter-sbmqf,node-exporter,NaN
2,1,node_network_info,00:00:00:00:00:00,00:00:00:00:00:00,kube-rbac-proxy,lo,https,master-2.sharedocp4upi411ovn.lab.upshift.rdu2....,node-exporter,openshift-monitoring,unknown,node-exporter-2vh2b,node-exporter,NaN
3,1,node_network_info,00:00:00:00:00:00,00:00:00:00:00:00,kube-rbac-proxy,lo,https,worker-0.sharedocp4upi411ovn.lab.upshift.rdu2....,node-exporter,openshift-monitoring,unknown,node-exporter-tcq6s,node-exporter,NaN
4,1,node_network_info,00:00:00:00:00:00,00:00:00:00:00:00,kube-rbac-proxy,lo,https,worker-1.sharedocp4upi411ovn.lab.upshift.rdu2....,node-exporter,openshift-monitoring,unknown,node-exporter-k5d5x,node-exporter,NaN
5,1,node_network_info,00:00:00:00:00:00,00:00:00:00:00:00,kube-rbac-proxy,lo,https,worker-2.sharedocp4upi411ovn.lab.upshift.rdu2....,node-exporter,openshift-monitoring,unknown,node-exporter-bmnhv,node-exporter,NaN
6,1,node_network_info,0a:75:c2:8d:b0:e6,ff:ff:ff:ff:ff:ff,kube-rbac-proxy,genev_sys_6081,https,master-2.sharedocp4upi411ovn.lab.upshift.rdu2....,node-exporter,openshift-monitoring,unknown,node-exporter-2vh2b,node-exporter,NaN
7,1,node_network_info,0e:3e:1b:fb:5f:a4,ff:ff:ff:ff:ff:ff,kube-rbac-proxy,genev_sys_6081,https,worker-2.sharedocp4upi411ovn.lab.upshift.rdu2....,node-exporter,openshift-monitoring,unknown,node-exporter-bmnhv,node-exporter,NaN
8,1,node_network_info,0e:51:58:e8:f8:84,ff:ff:ff:ff:ff:ff,kube-rbac-proxy,ovn-k8s-mp0,https,master-2.sharedocp4upi411ovn.lab.upshift.rdu2....,node-exporter,openshift-monitoring,unknown,node-exporter-2vh2b,node-exporter,NaN
9,1,node_network_info,12:83:97:59:aa:2a,ff:ff:ff:ff:ff:ff,kube-rbac-proxy,br-int,https,worker-2.sharedocp4upi411ovn.lab.upshift.rdu2....,node-exporter,openshift-monitoring,down,node-exporter-bmnhv,node-exporter,NaN


In [16]:
# https://prometheus.io/docs/prometheus/latest/querying/api/
r = requests.get(
    f"https://{prometheus_host}/v1/metadata",
    verify=False,
    headers=kubeConfig.api_key,
)
r.status_code, r.json()


(200,
 {'status': 'success',
  'data': {'aggregator_openapi_v2_regeneration_count': [{'type': 'counter',
     'help': '[ALPHA] Counter of OpenAPI v2 spec regeneration count broken down by causing APIService name and reason.',
     'unit': ''}],
   'aggregator_openapi_v2_regeneration_duration': [{'type': 'gauge',
     'help': '[ALPHA] Gauge of OpenAPI v2 spec regeneration duration in seconds.',
     'unit': ''}],
   'aggregator_unavailable_apiservice': [{'type': 'gauge',
     'help': '[ALPHA] Gauge of APIServices which are marked as unavailable broken down by APIService name.',
     'unit': ''}],
   'alertmanager_alerts': [{'type': 'gauge',
     'help': 'How many alerts by state.',
     'unit': ''}],
   'alertmanager_alerts_invalid_total': [{'type': 'counter',
     'help': 'The total number of received alerts that were invalid.',
     'unit': ''}],
   'alertmanager_alerts_received_total': [{'type': 'counter',
     'help': 'The total number of received alerts.',
     'unit': ''}],
   'al

In [17]:

r = requests.get(
    f"https://{prometheus_host}/v1/rules",
    verify=False,
    headers=kubeConfig.api_key,
)
r.status_code, r.json()


(200,
 {'status': 'success',
  'data': {'groups': [{'name': 'CloudCredentialOperator',
     'file': '/etc/prometheus/rules/prometheus-k8s-rulefiles-0/openshift-cloud-credential-operator-cloud-credential-operator-alerts-08a3b917-b260-4e67-acd0-3c15656d961d.yaml',
     'rules': [{'state': 'inactive',
       'name': 'CloudCredentialOperatorTargetNamespaceMissing',
       'query': 'cco_credentials_requests_conditions{condition="MissingTargetNamespace"} > 0',
       'duration': 300,
       'labels': {'severity': 'warning'},
       'annotations': {'description': "At least one CredentialsRequest custom resource has specified in its .spec.secretRef.namespace field a namespace which does not presently exist. This means the Cloud Credential Operator in the openshift-cloud-credential-operator namespace cannot process the CredentialsRequest resource. Check the conditions of all CredentialsRequests with 'oc get credentialsrequest -A' to find any CredentialsRequest(s) with a .status.condition showin

In [18]:

r = requests.get(
    f"https://{prometheus_host}/v1/labels",
    verify=False,
    headers=kubeConfig.api_key,
)
r.status_code, r.json()


(200,
 {'status': 'success',
  'data': ['From',
   'Local',
   'Remote',
   'To',
   '__name__',
   'access_mode',
   'action',
   'addr',
   'address',
   'alertmanager',
   'alertname',
   'alertstate',
   'apiserver',
   'apiservice',
   'approval',
   'attempts',
   'backend',
   'bios_date',
   'bios_vendor',
   'bios_version',
   'boot_id',
   'bound',
   'branch',
   'bridge',
   'broadcast',
   'build',
   'buildDate',
   'build_date',
   'build_phase',
   'build_user',
   'buildconfig',
   'cachesize',
   'call',
   'cause',
   'ceph_cluster',
   'ceph_daemon',
   'ceph_version',
   'certificatesigningrequest',
   'channel',
   'chassis_vendor',
   'chassis_version',
   'check',
   'checkName',
   'cidr',
   'claim_namespace',
   'client',
   'client_api_version',
   'clocksource',
   'cloud_type',
   'cluster_addr',
   'cluster_id',
   'cluster_ip',
   'cluster_version',
   'code',
   'collector',
   'command',
   'compiler',
   'completion_mode',
   'component',
   'compress

In [19]:
r = requests.get(
    f"https://{prometheus_host}/v1/label/system_uuid/values",
    verify=False,
    headers=kubeConfig.api_key,
)
r.json()

{'status': 'success',
 'data': ['3639c944-358b-40f7-b2f1-912b62d2333c',
  '651cac9e-d562-4040-a7c6-9692026388a8',
  '96f3e336-c833-4c1f-885f-9c97979d8fd1',
  '9ddbd942-0f1f-4ca8-948f-ce00f162f86a',
  'a3acce37-9195-428a-8aca-89c548c266c0',
  'd910f588-d8dc-44e7-86b7-68030155db51']}

In [20]:
r = requests.get(
    f"https://{prometheus_host}/v1/query?query=machine_cpu_cores&query=machine_cpu_sockets",
    verify=False,
    headers=kubeConfig.api_key,
)
r.json()

{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'__name__': 'machine_cpu_cores',
     'boot_id': '58f33e8d-1794-413c-843b-10288bb25356',
     'endpoint': 'https-metrics',
     'instance': '10.0.92.214:10250',
     'job': 'kubelet',
     'machine_id': 'd910f588d8dc44e786b768030155db51',
     'metrics_path': '/metrics/cadvisor',
     'namespace': 'kube-system',
     'node': 'master-2.sharedocp4upi411ovn.lab.upshift.rdu2.redhat.com',
     'service': 'kubelet',
     'system_uuid': 'd910f588-d8dc-44e7-86b7-68030155db51'},
    'value': [1671051626.318, '4']},
   {'metric': {'__name__': 'machine_cpu_cores',
     'boot_id': '87ada890-93b1-4036-b305-c79312f306e7',
     'endpoint': 'https-metrics',
     'instance': '10.0.92.177:10250',
     'job': 'kubelet',
     'machine_id': '651cac9ed5624040a7c69692026388a8',
     'metrics_path': '/metrics/cadvisor',
     'namespace': 'kube-system',
     'node': 'master-1.sharedocp4upi411ovn.lab.upshift.rdu2.redhat.com',
     

In [21]:
r = requests.get(
    f"https://{prometheus_host}/v1/metrics",
    verify=False,
    headers=kubeConfig.api_key,
)
r

<Response [404]>